# Data Aggregation


In [18]:
import pandas as pd

import sys
import os

## Aggregate survey results - English Cohorts

In [19]:
#
# aggregate all results from English cohorts in a single dataframe
#

path_results_en = "../private_data/data/survey_results_en/"

cohorts_en = [
    "2021.09",
    "2022.01",
    "2022.03",
    "2022.06",
    "2022.09",
    "2022.11",
    "2023.02",
    "2023.05",
    "2023.07",
    "2023.10",
    "2024.01",
    "2024.04",
    "2024.07",
    "2024.09",
    "2025.01",
]

dataframes_en = []

for cohort in cohorts_en:
    new_df = pd.read_excel(f"{path_results_en}/results_{cohort}.xlsx")
    dataframes_en.append(new_df)

merged_df_en = pd.concat(dataframes_en, ignore_index=True)

display(f"Number of entries: {len(merged_df_en) - 1}")


#
# Rename columns
#

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../scripts/aggregation/")))
from rename_columns_en import rename_columns_en

merged_df_en = rename_columns_en(merged_df_en)



#
# Map names to ct_student_id + remove names + remove comments
#
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../private_data/dictionaries_student_data/")))
from dictionary_students_en import dictionary_students_en

merged_df_en.insert(2, "ct_student_id", None) # add column "ct_student_id"
for student_name, cohort_id in dictionary_students_en.items():
    merged_df_en.loc[merged_df_en['name'].str.strip() == student_name.strip(), "ct_student_id"] = cohort_id

#
# Adding "cohort_language" column with "EN" value for English cohorts
#
merged_df_en.insert(4, "cohort_language", "EN")


#
# Remove columns that may contain personal data
#

merged_df_en.drop("name", axis=1, inplace=True)
merged_df_en.drop("final_comments", axis=1, inplace=True)
merged_df_en.drop("Marca temporal", axis=1, inplace=True)


#
# test
#
# pd.set_option('display.max_rows', None)
#display(merged_df_en)

'Number of entries: 70'

In [20]:
# Adding "other" job hunt blocker to a new column.

job_hunt_blockers_list = [
  "I never wanted to find a job as a developer",
  "I learned that web dev is not for me",
  "Lack of motivation",
  "Frustration with the process",
  "Feeling overwhelmed (too many things to do",
  # the next two relate to the previous entry with a ","
  "too many things to learn",
  "etc)",

  "Did't see many job offers",
  "Lack of confidence that I'd find a job",
  "Procrastination (I want",
  # the next one relate to the previous entry with a ","
  "but I keep delaying and postponing)",

  "Personal life (I'm focusing on other aspects of my personal life)",
  "Lack of guidance / don't know where to start",
]

# Function to find the unknown item
def find_unknown(blocker_str, valid_list):
    if pd.isna(blocker_str):
      return None
    items = [item.strip() for item in blocker_str.split(',')]
    unknowns = [item for item in items if item not in valid_list]
    #todo later, translate other
    return unknowns[0] if unknowns else None

# Apply the function to each row
merged_df_en['other_job_hunt_blocker'] = merged_df_en['job_hunt_blockers'].apply(lambda x: find_unknown(x, job_hunt_blockers_list))

In [21]:
# NOTE: removing commas and other characters and simplifying response for simpler EDA later. Repeating those not to be changed so the "other" is removed when not found.
job_hunt_blockers_dict = {
  "I never wanted to find a job as a developer": "I never wanted to find a job as a developer",
  "I learned that web dev is not for me": "I learned that web dev is not for me",
  "Lack of motivation": "Lack of motivation",
  "Frustration with the process": "Frustration with the process",
  "Feeling overwhelmed (too many things to do": "Feeling overwhelmed",
  # the next two relate to the previous entry with a ","
  "too many things to learn": "",
  "etc)": "",

  "Did't see many job offers": "Did not see many job offers",
  "Lack of confidence that I'd find a job": "Lack of confidence",
  "Procrastination (I want": "Procrastination",
  # the next one relate to the previous entry with a ","
  "but I keep delaying and postponing)": "",

  "Personal life (I'm focusing on other aspects of my personal life)": "Focusing on Personal life",
  "Lack of guidance / don't know where to start": "Lack of guidance",
}

# the lambda will split the substrings and apply the replace for each one
def filter_empty(elem):
  return elem != ""

merged_df_en['job_hunt_blockers'] = merged_df_en['job_hunt_blockers'].apply(
    lambda string: ', '.join(filter(filter_empty, [job_hunt_blockers_dict.get(substring.strip(), "Other") for substring in string.split(',')]))
    if pd.notnull(string) else string
)
# NOTE: adding "Other" instead of substring.strip() as a second argument of .get to group other responses. Later can be a different column.

In [22]:
# Adding "other" job success factor to a new column.

job_success_factor_list = [
  "Good communication and soft skills",
  "Good coding skills",
  "Strong portfolio or personal projects",
  "Networking and contacts",
  "I practiced a lot for technical interviews (e.g. coding challenges)",
  "I practiced a lot for non-technical interviews (e.g. behavioral questions",
  # the next three relate to the previous entry with a ","
  "explaining projects",
  "etc)",

  "Discipline and hard work",
  "Persistance (I got rejected many times but kept applying until I succeeded)",
]

# Function to find the unknown item
def find_unknown(factor_str, valid_list):
    if pd.isna(factor_str):
      return None
    items = [item.strip() for item in factor_str.split(',')]
    unknowns = [item for item in items if item not in valid_list]
    #todo later, translate other
    return unknowns[0] if unknowns else None

# Apply the function to each row
merged_df_en['other_job_success_factor'] = merged_df_en['job_success_factors'].apply(lambda x: find_unknown(x, job_success_factor_list))

In [23]:
# NOTE: removing commas and other characters and simplifying response for simpler EDA later. Repeating those not to be changed so the "other" is removed when not found.
job_success_factors_dict = {
  "Good communication and soft skills": "Good communication and soft skills",
  "Good coding skills": "Good coding skills",
  "Strong portfolio or personal projects": "Strong portfolio or personal projects",
  "Networking and contacts": "Networking and contacts",
  "I practiced a lot for technical interviews (e.g. coding challenges)": "I practiced a lot for technical interviews",
  "I practiced a lot for non-technical interviews (e.g. behavioral questions": "I practiced a lot for non-technical interviews",
  # the next three relate to the previous entry with a ","
  "explaining projects": "",
  "etc)": "",

  "Discipline and hard work": "Discipline and hard work",
  "Persistance (I got rejected many times but kept applying until I succeeded)": "Persistance",
}

# the lambda will split the substrings and apply the replace for each one

def filter_empty(elem):
  return elem != ""

merged_df_en['job_success_factors'] = merged_df_en['job_success_factors'].apply(
    lambda string: ', '.join(filter(filter_empty, [job_success_factors_dict.get(substring.strip(), "Other") for substring in string.split(',')]))
    if pd.notnull(string) else string
)

# NOTE: adding "Other" instead of substring.strip() as a second argument of .get to group other responses. Later can be a different column.

## Join results from English + Spanish cohorts

In [24]:
es_data_translated = pd.read_excel("../private_data/data/survey_results_es/es_data_translated.xlsx")

In [25]:
# NOTE: add cohort language if needed here
merged_df_en_es = pd.concat([es_data_translated, merged_df_en])

In [26]:
merged_df_en_es.to_excel("../private_data/data/survey_results_aggregated/data.xlsx", index=False)